In [1]:
# criar o navegador
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

servico = Service(ChromeDriverManager().install())
navegador = webdriver.Chrome(service=servico)

In [2]:
import os

caminho = os.getcwd()
arquivo = caminho + r"\index.html"
navegador.get(arquivo)

In [3]:
import pandas as pd
tabela = pd.read_excel("Processos.xlsx")
display(tabela)

,Nome,Advogado,Processo,Cidade,Status
0,Lira,Alon Lawyer,PC6592,Distrito Federal,NaN
1,João,Lawyer Alon,EB3792,Rio de Janeiro,NaN
2,Amanda,Amanda mesmo,MM1043,Rio de Janeiro,NaN
3,Carol,Amanda,PC5197,São Paulo,NaN


In [13]:
from selenium.webdriver import ActionChains
import time

for linha in tabela.index:
    
    # Abrir as opções
    objeto = navegador.find_element(By.XPATH, '/html/body/div/div/button')
    ActionChains(navegador).move_to_element(objeto).perform()
    
    # Escolher e clickar
    cidade = tabela.loc[linha, 'Cidade']
    navegador.find_element(By.PARTIAL_LINK_TEXT, cidade).click()
    
    # Trocar de navegador
    aba_principal = navegador.window_handles[0]
    outra_aba = navegador.window_handles[linha + 1]
    navegador.switch_to.window(outra_aba)
    
    # Preencher as info
    nome = tabela.loc[linha, 'Nome']
    advogado = tabela.loc[linha, 'Advogado']
    processo = tabela.loc[linha, 'Processo']
    navegador.find_element(By.NAME, 'nome').send_keys(nome)
    navegador.find_element(By.NAME, 'advogado').send_keys(advogado)
    navegador.find_element(By.NAME, 'numero').send_keys(processo)
    navegador.find_element(By.TAG_NAME, 'button').click()

    # Aceitar o alerta e verificar se existe processo
    alerta = navegador.switch_to.alert
    alerta.accept()
    
    while True:
        try:
            alerta = navegador.switch_to.alert
            break
        except:
            time.sleep(1)
    texto_alerta = alerta.text
    
    if 'Processo encontrado com sucesso!' in texto_alerta:
        alerta.accept()
        tabela.loc[linha, 'Status'] = 'Processo encontrado'
    else:
        alerta.accept()
        tabela.loc[linha, 'Status'] = 'Processo não encontrado'
    
    navegador.switch_to.window(aba_principal)

In [14]:
navegador.quit()
display(tabela)

tabela.to_excel("Processos atualizados.xlsx")

     Nome      Advogado Processo            Cidade                   Status
0    Lira   Alon Lawyer   PC6592  Distrito Federal  Processo não encontrado
1    João   Lawyer Alon   EB3792    Rio de Janeiro      Processo encontrado
2  Amanda  Amanda mesmo   MM1043    Rio de Janeiro  Processo não encontrado
3   Carol        Amanda   PC5197         São Paulo      Processo encontrado
